In [3]:
# Setting up data
Q = np.random.rand(6,6)
A = (Q.T + Q)/2 - np.eye(6)
A[0,0] += 3

# eigenvalues from max to min
eigenvalues, v = np.linalg.eig(A)
eig_vec_anal = v[:,0]
eig_val_anal = eigenvalues[0]
print(eig_vec_anal)
print(eig_val_anal)
print(eigenvalues)
print(v)

[-0.89864818 -0.17379462 -0.23178146 -0.17126443 -0.14984374 -0.23815873]
3.45773046233004
[ 3.45773046  0.47909152 -1.86961977 -1.61357221 -1.07654667 -0.88050535]
[[-0.89864818 -0.40223386 -0.03678202 -0.11215381 -0.12474307  0.03386944]
 [-0.17379462  0.16403305  0.08936799 -0.17406979  0.66993828 -0.67511804]
 [-0.23178146  0.44001724 -0.71343412  0.27813878  0.28926493  0.28746923]
 [-0.17126443  0.29471268  0.10269351  0.6450797  -0.46104649 -0.49454554]
 [-0.14984374  0.63400249  0.01236688 -0.64408398 -0.39913931 -0.03575455]
 [-0.23815873  0.35898633  0.68627495  0.22088018  0.28296928  0.46322378]]


In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
from tensorflow.keras.models import Sequential      #This allows appending layers to existing models
from tensorflow.keras.layers import Dense           #This allows defining the characteristics of a particular layer
from tensorflow.keras import optimizers             #This allows using whichever optimiser we want (sgd,adam,RMSprop)
from tensorflow.keras.regularizers import l2
tf.keras.backend.set_floatx('float64')

def loss(model, x, t):
    with tf.GradientTape() as tape_t:
        tape_t.watch([t])
        x_net = x + t / T_n * model(tf.concat([x,t], 1))
    dx_dt = tape_t.gradient(x_net, t)
    #rhs1 = -x + (tf.transpose(x) @ x @ A + (1 - tf.transpose(x) @ A @ x) @ np.eye(6)) @ x
    rhs = -x_net
    delta = tf.zeros((0, 1), dtype=tf.dtypes.float64)
    for i in range(Nt):
        x_i = x_net[6 * i : 6* i + 6]
        f_i = ( (tf.transpose(x_i) @ x_i)[0] * A + (1 - (tf.transpose(x_i) @ A @ x_i))[0] * tf.eye(6, dtype=tf.dtypes.float64) ) @ x_i
        delta = tf.concat([delta, f_i], 0)
    rhs += delta
    
    return tf.losses.mean_squared_error(zeros, dx_dt - rhs)

def train(model, optimizer, x, t):
    with tf.GradientTape() as tape:
        current_loss = loss(model, x, t)
        
    grads = tape.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

Nt = 10
Nx = 6
T_n = 1
x_np = np.random.rand(6,1)
t_np = np.linspace(0, T_n, Nt)

X, T = np.meshgrid(x_np, t_np)

x = X.ravel()
t = T.ravel()

zeros = tf.reshape(tf.convert_to_tensor(np.zeros(x.shape)), shape=(-1,1))
x = tf.reshape(tf.convert_to_tensor(x, dtype=tf.dtypes.float64), shape=(-1,1))
t = tf.reshape(tf.convert_to_tensor(t), shape=(-1,1))

# Setting up model
model = Sequential()
model.add(Dense(50, activation='sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(20, activation='sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(1, activation="linear"))
model.build(tf.concat([x,t], 1).shape)

eta = 0.00001
sgd = optimizers.SGD(lr=eta)

# Training model
num_iter = 100
for i in range(num_iter):
    train(model, sgd, x, t)
    
# Output of model
g_dnn = x + t * model(tf.concat([x,t], 1))
G_dnn = np.array(g_dnn).reshape((Nt, Nx))

In [179]:
for i in range(100):
    train(model, sgd, x, t)
    
# Output of model
g_dnn = x + t * model(tf.concat([x,t], 1))
G_dnn = np.array(g_dnn).reshape((Nt, Nx))

In [82]:
print(np.array(G_dnn[-1]))

[-12.63129355 -13.12418291 -12.88608573 -13.13447108 -13.01748744
 -13.19709608]


In [182]:
eig_vec = np.array(G_dnn[-1])
eig_vec /= np.linalg.norm(eig_vec)
print(eig_vec)
print(np.mean(A @ eig_vec / eig_vec))
print(A @ eig_vec / eig_vec)

[ 0.27316452  0.47477495 -0.15848149 -0.62384746 -0.19631829 -0.49711868]
0.6954586716361743
[ 1.54784572 -1.20928163  1.44289513  0.03374626  2.50341602 -0.14586946]


In [167]:
# eigenvalues from max to min
eigenvalues, v = np.linalg.eig(A)
eig_vec_anal = v[:,0]
eig_val_anal = eigenvalues[0]
print(eig_vec_anal)
print(eig_val_anal)

[0.83411571 0.26074626 0.29002581 0.23546724 0.23244084 0.20657647]
3.10068498098289


In [168]:
print(eigenvalues)
print(v)

[ 3.10068498  0.98605943 -0.41301284 -1.17878204 -1.08305551 -0.73561267]
[[ 0.83411571  0.53670782  0.09931789  0.05948915 -0.04724394  0.0236794 ]
 [ 0.26074626 -0.22551172 -0.64440465 -0.07367088  0.66411355  0.13937103]
 [ 0.29002581 -0.37310274 -0.36614215 -0.48022488 -0.60287311 -0.22033468]
 [ 0.23546724 -0.39054008  0.33876554  0.17923205  0.28140774 -0.75230119]
 [ 0.23244084 -0.43136612  0.56329662 -0.38849853  0.15732253  0.51663329]
 [ 0.20657647 -0.42811826 -0.09355937  0.75984371 -0.29887206  0.31400731]]


In [5]:
# symmetric matrix
Q = np.random.rand(6,6)
A = (Q.T + Q)/2
print(A)

[[0.6969513  0.67451162 0.79708163 0.89400824 0.43915108 0.54688166]
 [0.67451162 0.31661069 0.73883026 0.71930656 0.80375697 0.46555442]
 [0.79708163 0.73883026 0.58208262 0.78095336 0.53718359 0.61585948]
 [0.89400824 0.71930656 0.78095336 0.24548289 0.37394766 0.794089  ]
 [0.43915108 0.80375697 0.53718359 0.37394766 0.05636538 0.24077436]
 [0.54688166 0.46555442 0.61585948 0.794089   0.24077436 0.25773921]]


In [6]:
# eigenvalues from max to min
eigenvalues, v = np.linalg.eig(A)
print(eigenvalues)

[ 3.5952731   0.18753086 -0.71240732 -0.59003508 -0.08415091 -0.24097855]


In [7]:
def eigenvecsolver(xin, A, nits, dt): # xin is inputvector
    x = np.zeros((len(xin),nits))
    x[:,0] = xin
    for i in range(nits-1):
        x[:,i+1] = x[:,i] - x[:,i] * dt + (xin.T @ xin * A + (1 - xin.T @ A * xin) * np.eye(len(A))) @ xin * dt
    return x

In [8]:
xtry = np.random.rand(6)
nits = 100000
dt = 0.0001
x = eigenvecsolver(xtry,A,nits,dt)
t = np.linspace(0,nits,nits)

In [10]:
svar = x[:, -1]
print(svar)
print(np.mean(A @ svar / svar))
print(np.std(A @ svar / svar))

[7.81356128 8.89846428 9.92111161 7.04174324 4.76513138 6.44459996]
3.702346276096225
0.4990276817785955


In [ ]:
for i in range(len(x)):
    plt.plot(t,x[i,:])